In [75]:
import pandas as pd 
import numpy as np 
import statsmodels.api as sm 
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

In [64]:
# import data
g_data = pd.read_pickle('./data_max_1year')
print(len(g_data))
display(g_data.head())

249


,TDS,PMI_days,log_PMI_days,age_at_death,sex_male,est_weight_lb,est_stature_in,true_SOD_G_head,true_SOD_G_torso,true_SOD_G_limbs,fall,spring,summer,log_ADD_thres0,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,temp_1_3_mean,temp_1_3_std,hum_1_3_mean,hum_1_3_std,temp_4_7_mean,temp_4_7_std,hum_4_7_mean,hum_4_7_std,temp_8_21_mean,temp_8_21_std,hum_8_21_mean,hum_8_21_std,temp_22_56_mean,temp_22_56_std,hum_22_56_mean,hum_22_56_std,temp_57_154_mean,temp_57_154_std,hum_57_154_mean,hum_57_154_std,temp_155_365_mean,temp_155_365_std,hum_155_365_mean,hum_155_365_std,BMI,BMI_status
0,9.0,15.0,2.772589,77.0,0,180.0,64.000000,3.0,3.0,3.0,0,0,0,4.315902,73.881146,65.115868,32.350972,0.000000,0.000000,0.000000,0.00000,9.315,0.888,71.621,12.679,-3.108,7.418,62.885,6.736,-0.699,6.638,74.493,16.490,4.205,5.889,78.049,14.677,16.426,6.113,76.979,10.014,18.181,7.197,67.221,13.361,30.9,obese
1,13.0,79.0,4.382027,38.0,1,516.0,73.000039,5.0,5.0,3.0,0,1,0,7.190920,1326.323441,1326.323441,1279.608857,983.358857,391.756402,0.000000,0.00000,19.485,0.935,57.198,2.203,18.725,0.697,76.769,8.882,17.617,4.230,64.476,8.287,16.590,4.368,58.561,15.086,7.484,7.232,69.266,15.024,19.483,6.770,74.838,10.009,68.1,obese
2,15.0,164.0,5.105945,38.0,1,516.0,73.000039,6.0,6.0,3.0,0,0,1,8.175567,3552.068684,3552.068684,3505.354100,3209.104100,2617.501645,1785.331086,30.05625,27.174,1.174,71.959,6.483,28.471,0.317,63.152,0.949,26.955,1.031,72.737,6.045,26.864,1.680,67.719,7.432,18.606,5.250,63.088,13.413,12.155,8.104,73.635,12.741,68.1,obese
3,13.0,50.0,3.931826,73.0,1,235.0,73.000039,5.0,4.0,4.0,0,0,0,5.764677,317.835928,268.610789,126.581344,30.420863,0.000000,0.000000,0.00000,3.511,1.705,89.241,11.812,2.666,2.899,57.956,9.740,10.313,2.750,84.482,10.465,5.370,3.923,79.734,12.914,18.516,7.254,81.323,8.885,16.984,8.703,73.738,13.603,31.0,obese
4,6.0,11.0,2.484907,90.0,0,170.0,69.000037,2.0,2.0,2.0,0,0,0,4.258335,69.692188,54.013368,39.090451,15.414583,0.000000,0.000000,0.00000,3.825,1.623,80.427,17.534,3.942,4.081,64.240,4.184,6.799,5.078,79.896,12.073,10.311,4.085,80.913,11.039,24.042,2.560,80.822,7.902,12.899,9.807,73.032,14.609,25.1,overweight


In [65]:
def process_X(X, col_names_ls):
    # standardize numeric features
    X_stand = X.copy()

    # apply standardization
    for i in col_names_ls:
    
        # fit on training data column
        scale = StandardScaler().fit(X_stand[[i]])
    
        # transform the training data column
        X_stand[i] = scale.transform(X_stand[[i]])
        
    return X_stand

In [66]:
# create y dataframes
y_pmi = g_data[['log_PMI_days']]
display(y_pmi.head(2))
print(y_pmi.shape)

y_add = g_data[['log_ADD_thres0']]
display(y_add.head(2))
print(y_add.shape)

,log_PMI_days
0,2.772589
1,4.382027


(249, 1)


,log_ADD_thres0
0,4.315902
1,7.190920


(249, 1)


In [67]:
# create X dataframes
X1 = g_data[['TDS']]

X2 = g_data[['true_SOD_G_head', 'true_SOD_G_torso', 'true_SOD_G_limbs']]

X3 = g_data[['TDS', 'BMI', 'age_at_death', 'sex_male']]
X3_stand = process_X(X3, ['TDS', 'BMI', 'age_at_death'])
display(X3_stand.describe())

X4 = g_data[['TDS', 'BMI', 'age_at_death', 'sex_male',
            'fall', 'spring', 'summer', 'temp_1_3_mean', 'hum_1_3_mean', 'temp_4_7_mean',
           'hum_4_7_mean', 'temp_8_21_mean', 'hum_8_21_mean']]
X4_stand = process_X(X4, ['TDS', 'BMI', 'age_at_death', 'temp_1_3_mean', 'hum_1_3_mean', 'temp_4_7_mean',
           'hum_4_7_mean', 'temp_8_21_mean', 'hum_8_21_mean'])
display(X4_stand.describe())

X5 = g_data[['TDS',
            'fall', 'spring', 'summer', 'temp_1_3_mean', 'hum_1_3_mean', 'temp_4_7_mean',
           'hum_4_7_mean', 'temp_8_21_mean', 'hum_8_21_mean']]
X5_stand = process_X(X5, ['TDS', 'temp_1_3_mean', 'hum_1_3_mean', 'temp_4_7_mean',
           'hum_4_7_mean', 'temp_8_21_mean', 'hum_8_21_mean'])
display(X5_stand.describe())

X6 = g_data[['TDS',
            'fall', 'spring', 'summer', 'temp_8_21_mean', 'hum_8_21_mean']]
X6_stand = process_X(X6, ['TDS', 'temp_8_21_mean', 'hum_8_21_mean'])
display(X6_stand.describe())

,TDS,BMI,age_at_death,sex_male
count,2.490000e+02,2.490000e+02,2.490000e+02,249.000000
mean,1.070094e-16,-1.498132e-16,3.031934e-17,0.489960
std,1.002014e+00,1.002014e+00,1.002014e+00,0.500906
min,-2.389083e+00,-1.462613e+00,-2.353620e+00,0.000000
25%,-6.414871e-01,-7.189241e-01,-6.107163e-01,0.000000
50%,5.755136e-02,-2.128025e-01,-5.049721e-02,0.000000
75%,4.070706e-01,6.135184e-01,6.964615e-01,1.000000
max,2.504186e+00,3.991105e+00,2.003639e+00,1.000000


,TDS,BMI,age_at_death,sex_male,fall,spring,summer,temp_1_3_mean,hum_1_3_mean,temp_4_7_mean,hum_4_7_mean,temp_8_21_mean,hum_8_21_mean
count,2.490000e+02,2.490000e+02,2.490000e+02,249.000000,249.000000,249.000000,249.000000,2.490000e+02,2.490000e+02,2.490000e+02,2.490000e+02,2.490000e+02,2.490000e+02
mean,1.070094e-16,-1.498132e-16,3.031934e-17,0.489960,0.224900,0.325301,0.273092,-1.480297e-16,4.209038e-16,2.425547e-16,-7.633341e-16,5.350472e-18,1.269845e-15
std,1.002014e+00,1.002014e+00,1.002014e+00,0.500906,0.418357,0.469431,0.446445,1.002014e+00,1.002014e+00,1.002014e+00,1.002014e+00,1.002014e+00,1.002014e+00
min,-2.389083e+00,-1.462613e+00,-2.353620e+00,0.000000,0.000000,0.000000,0.000000,-2.955698e+00,-2.716764e+00,-2.695733e+00,-2.238968e+00,-2.226667e+00,-2.495596e+00
25%,-6.414871e-01,-7.189241e-01,-6.107163e-01,0.000000,0.000000,0.000000,0.000000,-7.058317e-01,-6.051202e-01,-7.730499e-01,-7.336664e-01,-9.038618e-01,-8.738093e-01
50%,5.755136e-02,-2.128025e-01,-5.049721e-02,0.000000,0.000000,0.000000,0.000000,1.903375e-01,3.759713e-02,2.182943e-01,7.102454e-02,2.918139e-02,8.625532e-02
75%,4.070706e-01,6.135184e-01,6.964615e-01,1.000000,0.000000,1.000000,1.000000,8.576176e-01,7.903372e-01,8.732316e-01,7.946061e-01,9.226394e-01,8.606300e-01
max,2.504186e+00,3.991105e+00,2.003639e+00,1.000000,1.000000,1.000000,1.000000,1.628336e+00,2.275239e+00,1.607551e+00,1.993054e+00,1.491550e+00,2.296210e+00


,TDS,fall,spring,summer,temp_1_3_mean,hum_1_3_mean,temp_4_7_mean,hum_4_7_mean,temp_8_21_mean,hum_8_21_mean
count,2.490000e+02,249.000000,249.000000,249.000000,2.490000e+02,2.490000e+02,2.490000e+02,2.490000e+02,2.490000e+02,2.490000e+02
mean,1.070094e-16,0.224900,0.325301,0.273092,-1.480297e-16,4.209038e-16,2.425547e-16,-7.633341e-16,5.350472e-18,1.269845e-15
std,1.002014e+00,0.418357,0.469431,0.446445,1.002014e+00,1.002014e+00,1.002014e+00,1.002014e+00,1.002014e+00,1.002014e+00
min,-2.389083e+00,0.000000,0.000000,0.000000,-2.955698e+00,-2.716764e+00,-2.695733e+00,-2.238968e+00,-2.226667e+00,-2.495596e+00
25%,-6.414871e-01,0.000000,0.000000,0.000000,-7.058317e-01,-6.051202e-01,-7.730499e-01,-7.336664e-01,-9.038618e-01,-8.738093e-01
50%,5.755136e-02,0.000000,0.000000,0.000000,1.903375e-01,3.759713e-02,2.182943e-01,7.102454e-02,2.918139e-02,8.625532e-02
75%,4.070706e-01,0.000000,1.000000,1.000000,8.576176e-01,7.903372e-01,8.732316e-01,7.946061e-01,9.226394e-01,8.606300e-01
max,2.504186e+00,1.000000,1.000000,1.000000,1.628336e+00,2.275239e+00,1.607551e+00,1.993054e+00,1.491550e+00,2.296210e+00


,TDS,fall,spring,summer,temp_8_21_mean,hum_8_21_mean
count,2.490000e+02,249.000000,249.000000,249.000000,2.490000e+02,2.490000e+02
mean,1.070094e-16,0.224900,0.325301,0.273092,5.350472e-18,1.269845e-15
std,1.002014e+00,0.418357,0.469431,0.446445,1.002014e+00,1.002014e+00
min,-2.389083e+00,0.000000,0.000000,0.000000,-2.226667e+00,-2.495596e+00
25%,-6.414871e-01,0.000000,0.000000,0.000000,-9.038618e-01,-8.738093e-01
50%,5.755136e-02,0.000000,0.000000,0.000000,2.918139e-02,8.625532e-02
75%,4.070706e-01,0.000000,1.000000,1.000000,9.226394e-01,8.606300e-01
max,2.504186e+00,1.000000,1.000000,1.000000,1.491550e+00,2.296210e+00


In [68]:
def fit_model(X, y):
    X = sm.add_constant(X) 
    est = sm.OLS(y, X).fit() 
    print(est.summary())
    
    y_pred = est.predict(X)
    rmse = mean_squared_error(y, y_pred, squared = False)
    print('RMSE:', rmse)
    
    y_pred_unlogged = np.exp(y_pred).copy()
    y_unlogged = np.exp(y).copy() - 1
    rmse_unlogged = mean_squared_error(y_unlogged, y_pred_unlogged, squared = False)
    print('RMSE_unlogged:', rmse_unlogged)

In [69]:
# TDS (baseline)
pmi1 = fit_model(X1, y_pmi)
print('############################################################################################')
add1 = fit_model(X1, y_add)

                            OLS Regression Results                            
Dep. Variable:           log_PMI_days   R-squared:                       0.265
Model:                            OLS   Adj. R-squared:                  0.262
Method:                 Least Squares   F-statistic:                     89.01
Date:                Mon, 22 Apr 2024   Prob (F-statistic):           3.03e-18
Time:                        13:48:37   Log-Likelihood:                -358.10
No. Observations:                 249   AIC:                             720.2
Df Residuals:                     247   BIC:                             727.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.4790      0.254      5.821      0.0

In [70]:
# weighted SODs 
pmi2 = fit_model(X2, y_pmi)
print('############################################################################################')
add2 = fit_model(X2, y_add)

                            OLS Regression Results                            
Dep. Variable:           log_PMI_days   R-squared:                       0.286
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     32.76
Date:                Mon, 22 Apr 2024   Prob (F-statistic):           7.70e-18
Time:                        13:48:38   Log-Likelihood:                -354.42
No. Observations:                 249   AIC:                             716.8
Df Residuals:                     245   BIC:                             730.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                1.5169      0.252  

In [71]:
# TDS plus demographics
pmi3 = fit_model(X3_stand, y_pmi)
print('############################################################################################')
add3 = fit_model(X3_stand, y_add)

                            OLS Regression Results                            
Dep. Variable:           log_PMI_days   R-squared:                       0.279
Model:                            OLS   Adj. R-squared:                  0.267
Method:                 Least Squares   F-statistic:                     23.55
Date:                Mon, 22 Apr 2024   Prob (F-statistic):           1.77e-16
Time:                        13:48:38   Log-Likelihood:                -355.77
No. Observations:                 249   AIC:                             721.5
Df Residuals:                     244   BIC:                             739.1
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            3.9066      0.091     42.852   

In [72]:
# TDS + demographics + weather
pmi4 = fit_model(X4_stand, y_pmi)
print('############################################################################################')
add4 = fit_model(X4_stand, y_add)

                            OLS Regression Results                            
Dep. Variable:           log_PMI_days   R-squared:                       0.388
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     11.46
Date:                Mon, 22 Apr 2024   Prob (F-statistic):           5.06e-19
Time:                        13:48:38   Log-Likelihood:                -335.28
No. Observations:                 249   AIC:                             698.6
Df Residuals:                     235   BIC:                             747.8
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.6599      0.227     20.

In [73]:
# TDS + weather
pmi5 = fit_model(X5_stand, y_pmi)
print('############################################################################################')
add5 = fit_model(X5_stand, y_add)

                            OLS Regression Results                            
Dep. Variable:           log_PMI_days   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.357
Method:                 Least Squares   F-statistic:                     14.75
Date:                Mon, 22 Apr 2024   Prob (F-statistic):           2.44e-20
Time:                        13:48:38   Log-Likelihood:                -336.37
No. Observations:                 249   AIC:                             694.7
Df Residuals:                     238   BIC:                             733.4
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.6121      0.218     21.

In [74]:
# TDS + weather features
pmi6 = fit_model(X6_stand, y_pmi)
print('############################################################################################')
add6 = fit_model(X6_stand, y_add)

                            OLS Regression Results                            
Dep. Variable:           log_PMI_days   R-squared:                       0.369
Model:                            OLS   Adj. R-squared:                  0.354
Method:                 Least Squares   F-statistic:                     23.60
Date:                Mon, 22 Apr 2024   Prob (F-statistic):           6.46e-22
Time:                        13:48:38   Log-Likelihood:                -339.05
No. Observations:                 249   AIC:                             692.1
Df Residuals:                     242   BIC:                             716.7
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              4.5383      0.210     21.